In [106]:
import os,re
import glob
import pandas

In [35]:
import os
def get_data(f_in):
    write_buf = []
    line = f_in.readline()
    slash_flag = 0
    while line:
        line = line.split("\n")[0]
        if line.find("//") >= 0:
            p = line.split("//")[0]  ## 存在返回起始坐标，从0开始，不存在，则返回-1
            if p != "" and (p.isspace() != True):
                write_buf.append(p)
        elif line.find("/*") >= 0 and line.find("*/") >= 0: 
            p = line.split("/*")[0]
            if p != "" and (p.isspace() != True): ## 排除只剩全空格或为空
                write_buf.append(p)
            p = line.split("*/")[-1]
            if p != "":
                write_buf.append(p)
        elif line.find("/*") >= 0:
            p = line.split("/*")[0]
            if p != "":
                write_buf.append(p)
            slash_flag = 1
        elif line.find("*/") >= 0:
            p = line.split("*/")[-1]
            if p != "" and p != "\n":
                write_buf.append(p)
            slash_flag = 0
        elif slash_flag == 1:
            line = f_in.readline()
            continue
        else:
            write_buf.append(line)       
        line = f_in.readline()
    return write_buf


In [128]:
# filename_dict = glob.glob("/root/data/qm/small-ase2022/data/Dataset-sard/No-Vul/*.c")
# filename_dict_Vul = glob.glob("/root/data/qm/small-ase2022/data/Dataset-sard/Vul/*.c")
ffmpeg_novul = glob.glob("/root/data/qm/small-ase2022/data/real_data/ffmpeg/NoVul/*.c")
ffmpeg_vul = glob.glob("/root/data/qm/small-ase2022/data/real_data/ffmpeg/Vul/*.c")
qemu_novul = glob.glob("/root/data/qm/small-ase2022/data/real_data/qemu/NoVul/*.c")
qemu_vul = glob.glob("/root/data/qm/small-ase2022/data/real_data/qemu/Vul/*.c")


In [129]:
len(ffmpeg_novul), len(ffmpeg_vul), len(qemu_novul), len(qemu_vul)

(4788, 4981, 10070, 7479)

In [120]:
len(filename_dict), len(filename_dict_Vul)

(21057, 12303)

In [130]:
final = []
for filename in ffmpeg_novul:
    f_in = open(filename, "r", encoding='UTF-8')
    gadget = [i.strip() for i in get_data(f_in) if i.strip() != ""]
    final.append({"gadget" : gadget, "val" : 0})
for filename in ffmpeg_vul:
    f_in = open(filename, "r", encoding='UTF-8')
    gadget = [i.strip() for i in get_data(f_in) if i.strip() != ""]
    final.append({"gadget" : gadget, "val" : 1})
for filename in qemu_novul:
    f_in = open(filename, "r", encoding='UTF-8')
    gadget = [i.strip() for i in get_data(f_in) if i.strip() != ""]
    final.append({"gadget" : gadget, "val" : 0})
for filename in qemu_vul:
    f_in = open(filename, "r", encoding='UTF-8')
    gadget = [i.strip() for i in get_data(f_in) if i.strip() != ""]
    final.append({"gadget" : gadget, "val" : 1})
df = pandas.DataFrame(final)

In [133]:
df.to_pickle("/root/data/qm/muVulDeePecker/pkl_data/real_all.pkl")

In [135]:
# df.to_pickle("/root/data/qm/muVulDeePecker/pkl_data/real_all_sub.pkl")
df_test = pandas.read_pickle("/root/data/qm/muVulDeePecker/pkl_data/real_all.pkl")
df_test

,gadget,val
0,[int ff_ivi_decode_frame(AVCodecContext *avctx...,0
1,"[int ffv1_init_slice_state(FFV1Context *f, FFV...",0
2,"[static void add_entry(TiffEncoderContext *s, ...",0
3,[static int check_bits_for_superframe(GetBitCo...,0
4,"[static int nprobe(AVFormatContext *s, uint8_t...",0
...,...,...
27313,"[bool migration_has_failed(MigrationState *s),...",1
27314,"[static int mode_sense_page(SCSIRequest *req, ...",1
27315,[static void exynos4210_uart_write(void *opaqu...,1
27316,[static DWORD WINAPI do_suspend(LPVOID opaque)...,1


In [91]:
filename = filename_dict[1]
filename = "/root/data/qm/small-ase2022/data/real_data/ffmpeg/NoVul/0_FFmpeg_019247bdc326a90bf20d3ce5d2413cc642e8bb0825450.c"
f_in = open(filename, "r", encoding='UTF-8')
gadget = [i.strip() for i in get_data(f_in) if i.strip() != ""]


In [122]:
gadget

['void CWE121_Stack_Based_Buffer_Overflow__CWE806_char_declare_memmove_64_bad()',
 '{',
 'char * data;',
 'char dataBuffer[100];',
 'data = dataBuffer;',
 "memset(data, 'A', 100-1);",
 "data[100-1] = '\\0';",
 'CWE121_Stack_Based_Buffer_Overflow__CWE806_char_declare_memmove_64b_badSink(&data);',
 '}']

In [85]:
text = " ".join(gadget)

In [86]:
text

'static int mov_read_stss(MOVContext *c, AVIOContext *pb, MOVAtom atom) { AVStream *st; MOVStreamContext *sc; unsigned int i, entries; if (c->fc->nb_streams < 1) return 0; st = c->fc->streams[c->fc->nb_streams-1]; sc = st->priv_data; avio_r8(pb); avio_rb24(pb); entries = avio_rb32(pb); av_dlog(c->fc, "keyframe_count = %d\\n", entries); if (!entries) { sc->keyframe_absent = 1; if (!st->need_parsing) st->need_parsing = AVSTREAM_PARSE_HEADERS; return 0; } if (entries >= UINT_MAX / sizeof(int)) return AVERROR_INVALIDDATA; sc->keyframes = av_malloc(entries * sizeof(int)); if (!sc->keyframes) return AVERROR(ENOMEM); for (i = 0; i < entries && !pb->eof_reached; i++) { sc->keyframes[i] = avio_rb32(pb); } sc->keyframe_count = i; if (pb->eof_reached) return AVERROR_EOF; return 0; }'

In [90]:
# tokenizer.decode(encoding['input_ids'][0]) 
tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

['<s>',
 'static',
 'Ġint',
 'Ġmov',
 '_',
 'read',
 '_',
 'st',
 'ss',
 '(',
 'M',
 'OV',
 'Context',
 'Ġ*',
 'c',
 ',',
 'ĠAV',
 'IO',
 'Context',
 'Ġ*',
 'pb',
 ',',
 'ĠMOV',
 'At',
 'om',
 'Ġatom',
 ')',
 'Ġ{',
 'ĠAV',
 'Stream',
 'Ġ*',
 'st',
 ';',
 'ĠMOV',
 'Stream',
 'Context',
 'Ġ*',
 'sc',
 ';',
 'Ġunsigned',
 'Ġint',
 'Ġi',
 ',',
 'Ġentries',
 ';',
 'Ġif',
 'Ġ(',
 'c',
 '->',
 'fc',
 '->',
 'nb',
 '_',
 'stream',
 's',
 'Ġ<',
 'Ġ1',
 ')',
 'Ġreturn',
 'Ġ0',
 ';',
 'Ġst',
 'Ġ=',
 'Ġc',
 '->',
 'fc',
 '->',
 'stream',
 's',
 '[',
 'c',
 '->',
 'fc',
 '->',
 'nb',
 '_',
 'stream',
 's',
 '-',
 '1',
 '];',
 'Ġsc',
 'Ġ=',
 'Ġst',
 '->',
 'priv',
 '_',
 'data',
 ';',
 'Ġav',
 'io',
 '_',
 'r',
 '8',
 '(',
 'pb',
 ');',
 'Ġav',
 'io',
 '_',
 'rb',
 '24',
 '(',
 'pb',
 ');',
 'Ġentries',
 'Ġ=',
 'Ġav',
 'io',
 '_',
 'rb',
 '32',
 '(',
 'pb',
 ');',
 'Ġav',
 '_',
 'd',
 'log',
 '(',
 'c',
 '->',
 'fc',
 ',',
 'Ġ"',
 'key',
 'frame',
 '_',
 'count',
 'Ġ=',
 'Ġ%',
 'd',
 '\\',
 'n',
 '"

In [84]:
from transformers import RobertaConfig, RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained('microsoft/codebert-base')

In [87]:
encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=256,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
)

In [124]:
cleaned_gadget = []
fun_symbols = {}
var_symbols = {}
fun_count = 1
var_count = 1
rx_comment = re.compile('\*/\s*$')
rx_fun = re.compile(r'\b([_A-Za-z]\w*)\b(?=\s*\()')
rx_var = re.compile(r'\b([_A-Za-z]\w*)\b(?:(?=\s*\w+\()|(?!\s*\w+))(?!\s*\()')
for line in gadget:
    print(line)

#     # process if not the header line and not a multi-line commented line
    if rx_comment.search(line) is None:
        nostrlit_line = re.sub(r'".*?"', '""', line)
        # remove all character literals
        nocharlit_line = re.sub(r"'.*?'", "''", nostrlit_line)
        # replace any non-ASCII characters with empty string
        ascii_line = re.sub(r'[^\x00-\x7f]', r'', nocharlit_line)
        user_fun = rx_fun.findall(ascii_line)
        user_var = rx_var.findall(ascii_line)
        print(ascii_line)
        for fun_name in user_fun:
            if len({fun_name}.difference(main_set)) != 0 and len({fun_name}.difference(keywords)) != 0:
                # DEBUG
                #print('comparing ' + str(fun_name + ' to ' + str(main_set)))
                #print(fun_name + ' diff len from main is ' + str(len({fun_name}.difference(main_set))))
                #print('comparing ' + str(fun_name + ' to ' + str(keywords)))
                #print(fun_name + ' diff len from keywords is ' + str(len({fun_name}.difference(keywords))))
                ###
                # check to see if function name already in dictionary
                if fun_name not in fun_symbols.keys():
                    fun_symbols[fun_name] = 'FUN' + str(fun_count)
                    fun_count += 1
                # ensure that only function name gets replaced (no variable name with same
                # identifier); uses positive lookforward
                ascii_line = re.sub(r'\b(' + fun_name + r')\b(?=\s*\()', fun_symbols[fun_name], ascii_line)

        for var_name in user_var:
            # next line is the nuanced difference between fun_name and var_name
            if len({var_name}.difference(keywords)) != 0 and len({var_name}.difference(main_args)) != 0:
                # DEBUG
                #print('comparing ' + str(var_name + ' to ' + str(keywords)))
                #print(var_name + ' diff len from keywords is ' + str(len({var_name}.difference(keywords))))
                #print('comparing ' + str(var_name + ' to ' + str(main_args)))
                #print(var_name + ' diff len from main args is ' + str(len({var_name}.difference(main_args))))
                ###
                # check to see if variable name already in dictionary
                if var_name not in var_symbols.keys():
                    var_symbols[var_name] = 'VAR' + str(var_count)
                    var_count += 1
                # ensure that only variable name gets replaced (no function name with same
                # identifier); uses negative lookforward
                ascii_line = re.sub(r'\b(' + var_name + r')\b(?:(?=\s*\w+\()|(?!\s*\w+))(?!\s*\()', \
                                    var_symbols[var_name], ascii_line)
        print(ascii_line)
        cleaned_gadget.append(ascii_line)
print(cleaned_gadget)

void CWE121_Stack_Based_Buffer_Overflow__CWE806_char_declare_memmove_64_bad()
void CWE121_Stack_Based_Buffer_Overflow__CWE806_char_declare_memmove_64_bad()


NameError: name 'main_set' is not defined

In [13]:
file = open(filename_dict[0],"rb")  # raw_000062516_goodB2G.c

In [41]:
with open(filename, "r", encoding="utf8") as file:
    data = file.read()  # 读取文件
    print(data)

static void goodB2G()
{
    int data;
    /* Initialize data */
    data = -1;
    {
#ifdef _WIN32
        WSADATA wsaData;
        int wsaDataInit = 0;
#endif
        int recvResult;
        struct sockaddr_in service;
        SOCKET connectSocket = INVALID_SOCKET;
        char inputBuffer[CHAR_ARRAY_SIZE];
        do
        {
#ifdef _WIN32
            if (WSAStartup(MAKEWORD(2,2), &wsaData) != NO_ERROR)
            {
                break;
            }
            wsaDataInit = 1;
#endif
            /* POTENTIAL FLAW: Read data using a connect socket */
            connectSocket = socket(AF_INET, SOCK_STREAM, IPPROTO_TCP);
            if (connectSocket == INVALID_SOCKET)
            {
                break;
            }
            memset(&service, 0, sizeof(service));
            service.sin_family = AF_INET;
            service.sin_addr.s_addr = inet_addr(IP_ADDRESS);
            service.sin_port = htons(TCP_PORT);
            if (connect(connectSocket, (struct sockaddr*)&servic

In [23]:
data

"static void goodG2B()\n{\n    char * data;\n    void (*funcPtr) (char *) = goodG2BSink;\n    char * dataBadBuffer = (char *)ALLOCA((10)*sizeof(char));\n    char * dataGoodBuffer = (char *)ALLOCA((10+1)*sizeof(char));\n    /* FIX: Set a pointer to a buffer that leaves room for a NULL terminator when performing\n     * string copies in the sinks  */\n    data = dataGoodBuffer;\n    data[0] = '\\0'; /* null terminate */\n    funcPtr(data);\n}\n"

In [ ]:
filename = "/root/data/qm/muVulDeePecker/mvd.txt"
save_path = "/root/data/qm/muVulDeePecker/pkl_data/"
if not os.path.exists(save_path): os.mkdir(save_path)
parse_file(filename)
base = os.path.splitext(os.path.basename(filename))[0]
df_filename = save_path + base + "_df.pkl"
# small_df_filename = save_path + base + "_small_df_200.pkl"
if os.path.exists(df_filename):
    df = pandas.read_pickle(df_filename)
else:
    df = get_df(filename)
    df.to_pickle(df_filename)
get_df_all(df, filename)